深入NumPy库的数值计算，包含ndarray类型（type）更多的细节描述和更高级的数组操作和算法。

**NumPymarkdown**

NumPy (short for Numerical Python)是高性能科学计算和数据分析的基础包。其部分功能如下：
+ ndarray，一个提供快速面向数组算术运算和复杂广播功能的高效多维数组
+ 数学函数——可以对整个数组进行快速运算，而不用写循环
+ 工具——读写磁盘数据以及操作内存映射文件
+ 线性代数、随机数生成和傅里叶变换
+ C API——用于将NumPy与用C、C++或者FORTRAN编写的库连接

NumPy在Python中对数值计算如此重要的原因之一是designed for efficiency on large arrays of data，原因如下：
+ NumPy internally stores data in a contiguous block of memeory, independent of other built-in Python objects. NumPy's library of algorithms written in the C language can operate on this memory without any type checking or other overhead.
+ NumPy arrays also use much less memory than built-in Python sequences.与Python序列相比，占用更少的内存。
+ Numpy operations perform complex computations on entire arrays without the need for Python for loops.对整个数组进行操作而不用写循环。

# 1 ndarray对象的内部机制（ndarray object internals）
NumPy的ndarray提供了一种将同质数据块（可以是连续或跨越的）解释为多维数组对象的方式。数据类型（dtype）决定了数据是怎样被解释为浮点数、整型、布尔型或其他任何类型。

ndarray这么灵活的一个原因是每个数组对象是一个数据块的跨度视图（strided view）。为什么数组视图`arr[::2, ::-1]`没有复制数据呢？这是因为ndarray不只是一块内存和一个dtype，它还有跨度（striding）信息，这使得数组能以各种步幅（step size）在内存中移动。更加准确地讲，ndarray内部由一下部分组成：

+ A *pointer* to data --that is, a block of data in RAM or in a memory-mapped file
+ The *data type* or *dtype*, describing fixed-size value celss in the array
+ A tuple indicating the array's shape
+ A tuple of strides, integers indicating the number of types to 'step' in order to advance one element along a dimension一个跨度元组，其中的整数指的是为了前进到当前维度下一个元素需要“跨过”的字节数。

下图简单地说明了ndarray内部结构：
![ndarray内部结构](https://raw.githubusercontent.com/libingallin/python-for-data-anlysis/master/numpy/figs/numpy-ndarry-object.png)

## 1.1 NumPy数据类型体系
有时需要check一个数组是否包含整数、浮点数，字符串或其他Python对象。因为这里有多种浮点数类型，check dtype是否属于某个大类比较繁琐。幸运地是，dtype都有一个超级类（如`np.integer`和`np.floating`），这可以与函数`np.issubdtype`联合使用：

In [1]:
import numpy as np

In [2]:
ints = np.ones(10, dtype=np.uint16)

In [3]:
np.issubdtype(ints.dtype, np.integer)

True

In [4]:
floats = np.ones(10, dtype=np.float32)

In [5]:
np.issubdtype(floats.dtype, np.floating)

True

**可以调用dtype的`mro`方法查看该dtype的所有父类：**

In [6]:
np.float64.mro()

[numpy.float64,
 numpy.floating,
 numpy.inexact,
 numpy.number,
 numpy.generic,
 float,
 object]

In [7]:
np.info(np.float32.mro)

mro() -> list
return a type's method resolution order


这些知识（特定dtype的父类）不需要必须掌握，但偶尔还是能派上用场的：
![dtype体系及其父子类关系](https://raw.githubusercontent.com/libingallin/python-for-data-anlysis/master/numpy/figs/numpy-dtype-class-hierarchy.png)

# 2 高级数组操作
有时需要编写现有库中找不到的数据算法。

## 2.1 重塑数组
很多时候需要**在不复制数据的同时，转换数组shape**。只需要向**`reshape`方法**传入一个新形状的元组即可：

In [8]:
arr = np.arange(8)
arr

array([0, 1, 2, 3, 4, 5, 6, 7])

In [9]:
arr.reshape((4, 2))

array([[0, 1],
       [2, 3],
       [4, 5],
       [6, 7]])

> 默认情况下，以C顺序。

In [10]:
arr.reshape((4, 2), order='F')

array([[0, 4],
       [1, 5],
       [2, 6],
       [3, 7]])

**传入的shape中元素可为-1，表示该维度的大小由数据推断而来。**

In [11]:
arr = np.arange(15)

In [12]:
arr.reshape((5, -1))

array([[ 0,  1,  2],
       [ 3,  4,  5],
       [ 6,  7,  8],
       [ 9, 10, 11],
       [12, 13, 14]])

**数据的`shape`属性是一个元组，因此可作为参数传入`reshape`中：**

In [13]:
other_arr = np.ones((3, 5))

In [14]:
other_arr.shape

(3, 5)

In [15]:
arr.reshape((other_arr.shape))

array([[ 0,  1,  2,  3,  4],
       [ 5,  6,  7,  8,  9],
       [10, 11, 12, 13, 14]])

**`reshape`的反向操作**——数组shape从多维到一维，称为**flattening/raveling。**

In [16]:
arr = np.arange(15).reshape((5, 3))
arr

array([[ 0,  1,  2],
       [ 3,  4,  5],
       [ 6,  7,  8],
       [ 9, 10, 11],
       [12, 13, 14]])

In [17]:
arr.ravel()

array([ 0,  1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14])

In [18]:
arr.flatten()

array([ 0,  1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14])

**`ravel`不会产生源数据的副本，如果values in the result were contiguous in the original array。`flatten`和`ravel`类似，但总是会返回数据的副本。**

## 2.2 C或Fortran顺序

numpy在控制数据在内存中的layout时有很大的灵活性。默认情况下，numpy数组以行优先（row major order）顺序创建的，这意味着每行数据在是被存放在相邻内存位置上。另一种是列优先顺序（column major order），这意味着每列数据是被存放相邻内存位置上。

因为一些历史的原因，行优先和列优先分别被称为C顺序和Fortran顺序。在FORTRAN77中，矩阵都是列优先的。

像`reshape`和`ravel`这样的函数都有一个`order`参数来表示数组存放位置。通常是C或F（也可以是A或K，具体参见文档）。

In [19]:
arr = np.arange(12).reshape((3, 4))
arr

array([[ 0,  1,  2,  3],
       [ 4,  5,  6,  7],
       [ 8,  9, 10, 11]])

In [20]:
arr.ravel()

array([ 0,  1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11])

In [21]:
arr.ravel('F')

array([ 0,  4,  8,  1,  5,  9,  2,  6, 10,  3,  7, 11])

二维或更多维数组的重塑有点难理解。C和Fortran关键的区别是在于维度的行进顺序：
+ **C/row major order**：更高的维度优先（如轴1优先于轴0）
+ **Fortran/column major order**：低维度的优先（如轴0在轴1前面）

## 2.3 数组的合并和拆分（Concatenating and Splitting arrays）

+ **`np.concatenate`**接收一个数组组成的sequence（tuple, list, etc.），然后沿着指定轴将他们连接到一起：

In [22]:
arr1 = np.array([[1, 2, 3], [4, 5, 6]])
arr2 = np.array([[7, 8, 9], [10, 11, 12]])

In [23]:
np.concatenate([arr1, arr2], axis=0)

array([[ 1,  2,  3],
       [ 4,  5,  6],
       [ 7,  8,  9],
       [10, 11, 12]])

In [24]:
np.concatenate([arr1, arr2], axis=1)

array([[ 1,  2,  3,  7,  8,  9],
       [ 4,  5,  6, 10, 11, 12]])

+ 对于常见的连接方式，有一些方便的函数——**`np.vstack`**和**`np.hstack`**。

In [25]:
np.vstack((arr1, arr2))

array([[ 1,  2,  3],
       [ 4,  5,  6],
       [ 7,  8,  9],
       [10, 11, 12]])

In [26]:
np.hstack((arr1, arr2))

array([[ 1,  2,  3,  7,  8,  9],
       [ 4,  5,  6, 10, 11, 12]])

+ **`np.split`**可以将数组沿指定轴切分为多个数组。

In [27]:
arr = np.random.randn(5, 2)
arr

array([[-1.81237955,  0.85745931],
       [ 1.56496973,  1.03947096],
       [-0.90853661,  1.2534361 ],
       [ 0.86707616, -1.57398313],
       [ 0.130598  ,  2.38642552]])

In [28]:
first, second, third = np.split(arr, [1, 3])

In [29]:
first

array([[-1.81237955,  0.85745931]])

In [30]:
first.shape

(1, 2)

In [31]:
second

array([[ 1.56496973,  1.03947096],
       [-0.90853661,  1.2534361 ]])

In [32]:
third

array([[ 0.86707616, -1.57398313],
       [ 0.130598  ,  2.38642552]])

传入的`[1, 3]`表示在哪个索引处切分数据。

**数组concatenate函数**

|函数|描述|
| :--- | :---- |
|concatenate|最常用的函数，沿指定轴连接一组数组|
|vstack, row_stack|面向行的方式对数组进行连接（沿轴0）|
|hstack, |面向列的方式对数组进行连接（沿轴1）|
|column_stack|类似于hstack，但是会先将1维数组转换为2维列向量|
|dstack|面向“深度”的方式对数组进行堆叠（沿轴2）
|split|沿特定的轴在指定位置切分数组|
|hsplit/vsplit|split的便捷化操作|

**stacking辅助：`r_`和`c_`**，更为简便的堆叠数组。

In [33]:
arr = np.arange(6)

In [34]:
arr1 = arr.reshape((3, 2))

In [35]:
arr2 = np.random.randn(3, 2)

In [36]:
arr1

array([[0, 1],
       [2, 3],
       [4, 5]])

In [37]:
arr2

array([[-0.83116129, -0.39413926],
       [ 0.13436007, -0.08740287],
       [ 1.06983337, -1.77300967]])

In [38]:
np.r_[arr1, arr2]

array([[ 0.        ,  1.        ],
       [ 2.        ,  3.        ],
       [ 4.        ,  5.        ],
       [-0.83116129, -0.39413926],
       [ 0.13436007, -0.08740287],
       [ 1.06983337, -1.77300967]])

In [39]:
np.c_[arr1, arr2]

array([[ 0.        ,  1.        , -0.83116129, -0.39413926],
       [ 2.        ,  3.        ,  0.13436007, -0.08740287],
       [ 4.        ,  5.        ,  1.06983337, -1.77300967]])

In [40]:
np.c_[np.r_[arr1, arr2], arr]

array([[ 0.        ,  1.        ,  0.        ],
       [ 2.        ,  3.        ,  1.        ],
       [ 4.        ,  5.        ,  2.        ],
       [-0.83116129, -0.39413926,  3.        ],
       [ 0.13436007, -0.08740287,  4.        ],
       [ 1.06983337, -1.77300967,  5.        ]])

In [41]:
np.c_[1:6, -10:-5]

array([[  1, -10],
       [  2,  -9],
       [  3,  -8],
       [  4,  -7],
       [  5,  -6]])

## 2.4 元素的重复操作：tile和repeat

**title**和**tile**是两个最有用的函数——来重复数组以产生更大的数组。

+ **title**将数组中的每个元素重复(replicate)一定的次数，来产生更大的数组：

In [42]:
arr = np.arange(3)
arr

array([0, 1, 2])

In [43]:
arr.repeat(3)

array([0, 0, 0, 1, 1, 1, 2, 2, 2])

> NumPy中很少需要对数组进行重复，这是因为**broadcasting**可以更好的满足这一需求。

默认情况下，如果传入一个整数，则数组中的每个元素都会被重复那么多次。如果传入一个整数数组（an array of integers），则各个元素会被重复不同的次数。

In [44]:
arr.repeat([2, 3, 4])

array([0, 0, 1, 1, 1, 2, 2, 2, 2])

对于多维数组，还可以沿指定轴对元素进行重复。

In [45]:
arr = np.arange(4).reshape((2, 2))
arr

array([[0, 1],
       [2, 3]])

In [46]:
arr.repeat(2, axis=0)

array([[0, 1],
       [0, 1],
       [2, 3],
       [2, 3]])

In [47]:
arr.repeat(2, axis=1)

array([[0, 0, 1, 1],
       [2, 2, 3, 3]])

如果没有指定轴，那么数组首先会被扁平化（flatten）。

In [48]:
arr.repeat(2)

array([0, 0, 1, 1, 2, 2, 3, 3])

对于多维数组重复时，也可以传入一组整数，这样的话就会使各切片重复不同的次数。

In [49]:
arr.repeat([2, 3], axis=0)

array([[0, 1],
       [0, 1],
       [2, 3],
       [2, 3],
       [2, 3]])

In [50]:
arr.repeat([2, 3], axis=1)

array([[0, 0, 1, 1, 1],
       [2, 2, 3, 3, 3]])

**tile**沿指定轴堆叠数组的副本。可以理解为“铺瓷砖”。

In [51]:
arr

array([[0, 1],
       [2, 3]])

In [52]:
np.tile(arr, 2)

array([[0, 1, 0, 1],
       [2, 3, 2, 3]])

第二个元素是“瓷砖”的数量。如果是标量，则水平铺，而不是垂直。第二个参数也可以是表示铺设布局的元组。

In [53]:
arr

array([[0, 1],
       [2, 3]])

In [54]:
np.tile(arr, (2, 1))

array([[0, 1],
       [2, 3],
       [0, 1],
       [2, 3]])

In [55]:
np.tile(arr, (3, 2))

array([[0, 1, 0, 1],
       [2, 3, 2, 3],
       [0, 1, 0, 1],
       [2, 3, 2, 3],
       [0, 1, 0, 1],
       [2, 3, 2, 3]])

> 沿轴0重复3次，沿轴1重复2次。元组的shape和数组的shape一样。

## 2.5 花式索引的等价函数——`take`和`put`

获取数组子集的一个方法是通过整数数组使用花式索引。

In [56]:
arr = np.arange(10) * 100
arr

array([  0, 100, 200, 300, 400, 500, 600, 700, 800, 900])

In [57]:
inds = [7, 1, 2, 6]

In [58]:
arr[inds]

array([700, 100, 200, 600])

+ `take`——沿单个轴选择数组子集：

In [59]:
arr.take(inds)

array([700, 100, 200, 600])

+ `put`——给数组子集赋值：

In [60]:
arr.put(inds, 42)

In [61]:
arr

array([  0,  42,  42, 300, 400, 500,  42,  42, 800, 900])

In [62]:
arr.put(inds, [40, 41, 42, 43])
arr

array([  0,  41,  42, 300, 400, 500,  43,  40, 800, 900])

> `put`原地操作。`put`没有`axis`参数，只会在扁平化的数组上进行索引。因此，需要在其他轴上进行设置元素时，最好还是使用花式索引。

`take`沿其他轴进行操作：

In [64]:
arr = np.arange(8).reshape((2, 4))
arr

array([[0, 1, 2, 3],
       [4, 5, 6, 7]])

In [65]:
inds = [2, 0, 2, 1]

In [66]:
arr.take(inds, axis=1)

array([[2, 0, 2, 1],
       [6, 4, 6, 5]])

# 3 广播（broadcasting）

广播是指不同shape的数组之间的算术运算执行方式。功能强大，但容易引起困惑。

**最简单的广播发生在将标量和数组合并时。**

In [68]:
arr = np.arange(5)
arr

array([0, 1, 2, 3, 4])

In [69]:
arr * 4

array([ 0,  4,  8, 12, 16])

> 这里，可以说：在乘法操作中，标量值4被广播到其他所有元素上。

可以通过减去列平均值的方式对数组的每一列进行距平化处理:

In [70]:
arr = np.random.randn(4, 3)
arr

array([[-0.75236963, -1.22186617, -0.15407726],
       [-0.22739966,  1.2108285 ,  0.4724713 ],
       [ 0.7535036 ,  0.465045  , -1.54424534],
       [ 0.33225264, -1.95410008, -0.39267789]])

In [71]:
arr.mean(0)

array([ 0.02649674, -0.37502319, -0.4046323 ])

In [72]:
demeaned = arr - arr.mean(0)
demeaned

array([[-0.77886637, -0.84684298,  0.25055503],
       [-0.2538964 ,  1.58585169,  0.8771036 ],
       [ 0.72700687,  0.84006819, -1.13961304],
       [ 0.3057559 , -1.5790769 ,  0.01195441]])

In [73]:
demeaned.mean(0)

array([-2.77555756e-17,  0.00000000e+00,  8.32667268e-17])

In [ ]:
只要遵循一定的规则，低维度的值可以